## hw3 LSH索引

在corel数据集上实现LSH索引

并分别进行近邻搜索，查询点为数据集前1000点，查找前10个最近邻

统计搜索算法的性能（召回率，准确率，时间）。

In [1]:
import pandas as pd 
import numpy as np
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors

In [2]:
df = pd.read_csv("ColorHistogram.asc", header=None, sep=' ')
data = np.array(df)[:, 1:]
print(data.shape)
df.iloc[:10, 1:]

(68040, 32)


,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
0,0.002188,0.000000,0.000000,0.620521,0.010313,0.007083,0.043021,0.310729,0.000729,0.000000,...,0.000000,0.000000,0.000937,0.000000,0.000000,0.000000,0.000417,0.000000,0.000000,0.000000
1,0.002917,0.315417,0.188854,0.004440,0.000001,0.000001,0.000004,0.000032,0.000000,0.000000,...,0.000000,0.000000,0.000208,0.001563,0.003750,0.002708,0.007917,0.326562,0.133958,0.011771
2,0.000313,0.009825,0.008978,0.663125,0.002083,0.003542,0.006250,0.009688,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000417,0.000000,0.000000,0.000834,0.001701,0.004701,0.288647
3,0.111667,0.123855,0.078230,0.085486,0.015104,0.026563,0.015938,0.004064,0.003958,0.012708,...,0.000417,0.000000,0.053229,0.003854,0.000104,0.000000,0.117604,0.010209,0.000834,0.000030
4,0.329803,0.522930,0.034487,0.011571,0.005835,0.000315,0.000314,0.003542,0.000834,0.000418,...,0.000521,0.000521,0.000318,0.000313,0.000208,0.000312,0.015950,0.040522,0.006979,0.000417
5,0.153368,0.242450,0.246149,0.102189,0.007709,0.007083,0.005104,0.017084,0.000417,0.001042,...,0.000000,0.000000,0.006146,0.001458,0.000000,0.000000,0.015835,0.011983,0.001772,0.000000
6,0.002099,0.006171,0.090442,0.395948,0.000417,0.000938,0.001146,0.000523,0.000208,0.004792,...,0.000000,0.000000,0.001459,0.002188,0.000312,0.000000,0.003022,0.009376,0.098438,0.368125
7,0.099375,0.193334,0.197301,0.031187,0.000104,0.000000,0.002083,0.016667,0.000104,0.000000,...,0.000000,0.000000,0.085313,0.007500,0.000000,0.000000,0.185104,0.067293,0.013239,0.000351
8,0.002094,0.008249,0.292416,0.194587,0.000833,0.003958,0.025521,0.027710,0.000313,0.000625,...,0.000000,0.000000,0.000113,0.005313,0.003542,0.002292,0.000540,0.121153,0.268022,0.039896
9,0.136459,0.235105,0.000030,0.000020,0.003125,0.010625,0.000003,0.000003,0.000000,0.000000,...,0.004896,0.000208,0.014688,0.000001,0.000000,0.000000,0.040417,0.002929,0.000028,0.000004


In [3]:
n_qs = 1000      # 查找数据集前1000点
k = 10           # 查找10个最近邻

In [ ]:
# 使用KNN生成groundtruth
nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(data)
distances, indices = nbrs.kneighbors(data)
groundtruth = indices[:n_qs]

In [ ]:
np.savetxt('indices_{}.csv'.format(k), indices[:n_qs], delimiter = ',')
np.savetxt('distances_{}.csv'.format(k), distances[:n_qs], delimiter = ',')

In [4]:
df = pd.read_csv('indices_{}.csv'.format(k), header=None, sep=',')
groundtruth = np.array(df, dtype=np.int64)[:n_qs]

In [5]:
# 归一化，将范围缩放到 [0,1]
# min_max_scaler = preprocessing.MinMaxScaler()
# data = min_max_scaler.fit_transform(data)

![](./pic.png)

$a$是稳定生成的随机向量

$W$是直线上分段的段长

$b$是$(0,W)$里的随机向量/矩阵

$o$是待计算特征向量/矩阵，即特征

$h_k(o)$表示落入的桶的ID（这里ID的范围从0至999）

$$h_k(\mathbf{o})=\lfloor\frac{\Vert{\mathbf{a} \cdot \mathbf{o}+\mathbf{b}}\Vert}{W}\rfloor$$

In [6]:
def h_k(o, w):
    a = np.random.rand(data.shape[1])
    b = w * np.random.rand(data.shape[0] * data.shape[1]).reshape(data.shape)
    return np.linalg.norm(a * o + b, axis=1) // w

一个LSH函数族含有$k$个哈希函数定义为
$$H(o)=(h_1(o), h_1(o), \dots , h_k(o))$$
若$h_i(x)=h_i(y)$，则说明$x$和$y$落入同一个“桶”中，$x$和$y$落入相同“桶”的次数越多，表明$x$和$y$越相似。


In [7]:
# 最大模长
max_len = np.linalg.norm([np.max(data[:, _]) for _ in range(data.shape[1])])

# 最多n_bucket个桶
n_bucket = 256

# 根据n_bucket确定段长
W = max_len / n_bucket

def H(o, k_h):
    return np.array([h_k(o, W) for _ in range(k_h)]).T

In [8]:
%%time
k_h = 256      # k越大，每组中使用的哈希函数越多，结果越逼近p范数近邻的结果

# H1(data, k_h, l_h)
data_hash = H(data, k_h)
# print(s)
prediction = np.zeros((n_qs, k), dtype=np.int64)
for i in range(n_qs):
    cnt = np.sum(data_hash == data_hash[i], axis=1)
    index = np.argsort(cnt)
    prediction[i] = index[-1*k :][::-1]
print(prediction)
print(groundtruth)

[[    0 64186 64170 ... 45898 24932 46034]
 [    1 45827    80 ... 41161 56641 47645]
 [    2 40317  7133 ... 51511 60094 50572]
 ...
 [  997 33625 58862 ... 30989 43119  4440]
 [  998 23845  3402 ... 66591 34239  3444]
 [  999 22131 31147 ... 18047  8890 35625]]
[[    0 33344 64186 ... 56634 52696 64170]
 [    1 56641    80 ... 29328 14697 41161]
 [    2 40317  7133 ... 58688 46207  9357]
 ...
 [  997  4448  4360 ... 57731 43119 62136]
 [  998 45405  3423 ...   911 47517  8810]
 [  999   492 66926 ...   493 35850 12811]]
Wall time: 3min 56s


![](./pic2.png)

前$k$个近邻为正，其他为负，计算准确率和召回率

In [9]:
%%time
acc = [0 for _ in range(n_qs)]
recall = [0 for _ in range(n_qs)]
precision = [0 for _ in range(n_qs)]

all_i = set([i for i in range(data.shape[0])])

for i in range(n_qs):
    positive = set(groundtruth[i])
    negative = all_i - positive
    positive_p = set(prediction[i])
    
    tp = len(positive_p - negative)
    fn = len(positive - positive_p)
    fp = len(positive_p - positive)
    tn = len(negative - positive_p)
    
    acc[i] = (tp + tn) / len(all_i)
    recall[i] = tp / (tp + fn)
    precision[i] = tp / (tp + fp)

acc = np.mean(acc)
recall = np.mean(recall)
precision = np.mean(precision)
F = 2 * recall * precision / (recall + precision)
print("数据总量为{}，查询数据集前{}点，前{}个最近邻".format(data.shape[0], n_qs, k))
print("准确率：{}\n召回率：{}\n精确率：{}\nF值：{}".format(acc, recall, precision, F))

数据总量为68040，查询数据集前1000点，前10个最近邻
准确率：0.9998197236919459
召回率：0.3867
精确率：0.3867
F值：0.38669999999999993
Wall time: 24.1 s
